# Mistral-7B Fine-tuning for Emission Factor Recommendations

This notebook implements the fine-tuning process for the Mistral-7B model to generate emission factor recommendations based on the PRD specifications.

## Prerequisites

1. GPU Runtime in Google Colab
2. Google Drive mounted for saving checkpoints
3. Hugging Face account with access to Mistral-7B-Instruct-v0.2
4. HF_TOKEN in Colab secrets
5. Weights & Biases account for experiment tracking

## 1. Environment Setup

Install required packages and mount Google Drive.

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install required packages - updated versions for compatibility
!pip install -q torch==2.1.0
!pip install -q huggingface_hub==0.19.4
!pip install -q accelerate==0.25.0
!pip install -q transformers==4.35.0
!pip install -q peft==0.7.1
!pip install -q datasets scipy wandb trl==0.7.4 bitsandbytes==0.40.2

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

## 2. Hugging Face Authentication

Authenticate with Hugging Face and verify access to the model.

In [2]:
import os
from huggingface_hub import login, HfApi

# Login to Hugging Face
login(token=os.environ.get('HF_TOKEN'))

# Verify access
api = HfApi()
try:
    api.model_info("mistralai/Mistral-7B-Instruct-v0.2")
    print("Successfully authenticated and have access to the model!")
except Exception as e:
    print(f"Error: {e}")

Successfully authenticated and have access to the model!


## 3. Weights & Biases Setup

Initialize Weights & Biases for experiment tracking.

In [3]:
import wandb

# Initialize wandb - this will open a browser tab for authentication if not already logged in
# If you're already logged in through the browser, this will use your existing session
wandb.login()

# Define project and experiment details
WANDB_PROJECT = "mistral-carbon-ef"
EXPERIMENT_NAME = "mistral-7b-emission-factors"

# Initialize the wandb run
wandb.init(
    project=WANDB_PROJECT,
    name=EXPERIMENT_NAME,
    config={
        "model_name": "mistralai/Mistral-7B-Instruct-v0.2",
        "epochs": 3,
        "learning_rate": 2e-4,
        "batch_size": 1,
        "gradient_accumulation_steps": 8,
        "lora_rank": 8,
        "lora_alpha": 16
    }
)

print("Weights & Biases initialized successfully!")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: allaitest2024 (allaitest2024-yeshiva-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Weights & Biases initialized successfully!


## 4. Model and Tokenizer Configuration

Set up the Mistral-7B model with 16-bit precision and LoRA configuration.

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

# Model configuration
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Explicitly set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Fixed model loading to avoid meta tensor error
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map=None,  # Don't use auto device mapping
    trust_remote_code=True,
)
model = model.to(device)  # Explicitly move to device

# Configure LoRA
lora_config = LoraConfig(
    r=8,  # Reduced rank for better memory usage
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA
model = get_peft_model(model, lora_config)
print("Model and tokenizer configured successfully!")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

RuntimeError: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
module 'torch.library' has no attribute 'register_fake'

## 5. Data Preparation

Load and prepare the training data. If the data files don't exist, create sample data for testing.

In [ ]:
# Clone the repository
!git clone https://github.com/Sbursu/Carbon-EF.git
%cd Carbon-EF

# Create sample data if it doesn't exist
import os
import json
from datasets import load_dataset

# Check for the data directory and create it if it doesn't exist
!mkdir -p training/data

# Define sample data for emission factors
sample_data = [
    {
        "instruction": "Recommend an emission factor for rice production in Asia.",
        "output": "For rice production in Asia, I recommend using an emission factor of 1.46 kg CO2e per kg of rice, based on the Agribalyse database."
    },
    {
        "instruction": "What is the carbon footprint of beef production?",
        "output": "The carbon footprint of beef production is approximately 27 kg CO2e per kg of beef, making it one of the highest emission foods."
    },
    {
        "instruction": "Provide an emission factor for electricity generation from coal.",
        "output": "For electricity generation from coal, the emission factor is approximately 1000 g CO2e per kWh, which is significantly higher than renewable sources."
    }
]

# Create train, validation, and test files if they don't exist
data_files = {
    'training/data/instructions_train.json': sample_data[:2],
    'training/data/instructions_val.json': [sample_data[2]],
    'training/data/instructions_test.json': [sample_data[2]]
}

for file_path, data in data_files.items():
    if not os.path.exists(file_path):
        print(f"Creating sample data file: {file_path}")
        with open(file_path, 'w') as f:
            json.dump(data, f)

# Now load the datasets
try:
    train_data = load_dataset('json', data_files='training/data/instructions_train.json')
    val_data = load_dataset('json', data_files='training/data/instructions_val.json')
    test_data = load_dataset('json', data_files='training/data/instructions_test.json')

    # Format instruction template
    def format_instruction(example):
        instruction = example['instruction']
        input_text = example.get('input', '')
        output = example['output']

        if input_text:
            formatted = f"<s>[INST] {instruction}\n\n{input_text} [/INST] {output} </s>"
        else:
            formatted = f"<s>[INST] {instruction} [/INST] {output} </s>"

        return {'text': formatted}

    # Apply formatting
    train_data = train_data.map(format_instruction)
    val_data = val_data.map(format_instruction)
    test_data = test_data.map(format_instruction)

    print(f"Training samples: {len(train_data['train'])}")
    print(f"Validation samples: {len(val_data['train'])}")
    print(f"Test samples: {len(test_data['train'])}")

except Exception as e:
    print(f"Error loading datasets: {e}")
    print("Please check that the data files exist and are properly formatted.")

## 6. Training Configuration

Set up training arguments and initialize the trainer.

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/mistral-ef-checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=1,  # Reduced batch size for better memory usage
    gradient_accumulation_steps=8,  # Increased gradient accumulation steps
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",  # Fixed deprecated warning
    load_best_model_at_end=True,
    report_to="wandb",  # Log metrics to Weights & Biases
    run_name=EXPERIMENT_NAME,  # Use the same experiment name for wandb
    warmup_ratio=0.1,
    gradient_checkpointing=True,  # Enable gradient checkpointing to save memory
    optim="adamw_torch",
    max_grad_norm=0.3
)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Initialize trainer with fixed parameters (avoid deprecated warning)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data['train'],
    eval_dataset=val_data['train'],
    data_collator=data_collator
)

print("Training configuration completed!")

## 7. Training Process

Start the fine-tuning process.

In [ ]:
# Start training
trainer.train()

# Save the final model
trainer.save_model("/content/drive/MyDrive/mistral-ef-final")

## 8. Model Evaluation

Evaluate the fine-tuned model on the test set.

In [ ]:
# Evaluate on test set
test_results = trainer.evaluate(test_data['train'])
print(f"Test results: {test_results}")

# Log test results to wandb
wandb.log({"test_loss": test_results["loss"]})

## 9. Save and Export

Save the model and tokenizer to Google Drive.

In [ ]:
# Save model and tokenizer
model.save_pretrained("/content/drive/MyDrive/mistral-ef-final")
tokenizer.save_pretrained("/content/drive/MyDrive/mistral-ef-final")

# Save training configuration
import json
with open("/content/drive/MyDrive/mistral-ef-final/training_config.json", "w") as f:
    json.dump({
        "model_name": MODEL_NAME,
        "lora_config": lora_config.to_dict(),
        "training_args": training_args.to_dict(),
        "test_results": test_results
    }, f, indent=2)

# Finish wandb run
wandb.finish()

print("Model and configuration saved successfully!")